In [ ]:
!pip install -q langchain-community
!pip install -q langchain-core
!pip install -q transformers[torch]
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu
!pip install -q pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.6/973.6 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.4/310.4 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/20240530_resources_generalitve-ai-guidebook_01.pdf")
pages = loader.load_and_split()

In [ ]:
print(pages)

[Document(page_content='テキスト生成 AI利活用におけるリスクへの対策ガイ\nドブック（α版）  \n2024（令和6）年 5月30日 \nデジタル庁', metadata={'source': '/content/20240530_resources_generalitve-ai-guidebook_01.pdf', 'page': 0}), Document(page_content='〔ドキュメントの位置付け〕  \n参考資料。 今後、デジタル社会推進標準ガイドラインへの編入を 検討予定  \n \n〔キーワード〕  \nテキスト生成 AI、生成AI、サービス開発者、サービス提供者  \n \n〔概要〕  \nテキスト生成 AIを利活用し、行政サービスや職員業務の改善の重要度が高まる中、リ\nスクを特定し、そのリスクを受容できるレベルまでに軽減する対応もまた重要になってい\nる。テキスト生成 AIに関連するリスクは多岐にわたるが、その多くはテキスト生成 AI固有\nでない AIシステム全般に共通するものである。そこで、本文書では政府情報システムを対\n象に、テキスト生成 AI固有と見られるリスクに焦点をあて、留意点を紹介する。現段階\n（2024年5月現在）では、実践的なフレームワークやチェックリストによるガイドブックを作\n成できるまでの知見が蓄積されていない ため、留意点の紹介にとどめる。本文書は、テキ\nスト生成 AIをその提供形態と利活用方法で分類し、それぞれにおける想定リスクとその軽\n減策を具体的に示したものである', metadata={'source': '/content/20240530_resources_generalitve-ai-guidebook_01.pdf', 'page': 1}), Document(page_content='改定履歴  \n \n改定年月日  改定箇所  改定内容  \n2024年5月30日 複数 誤記等の軽微な修 正 \n2024年5月29日 - 初版決定', metadata={'source': '/content/20240530_resources_generalitve-ai-guidebook_01.pdf', 'page': 2}), Document(page_

In [ ]:
# RecursiveCharacterTextSplitterのインスタンスを作成します。チャンクサイズ1000,チャンクの重複サイズは150
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

# ドキュメントを分割します
docs = text_splitter.split_documents(pages)

In [ ]:
docs[0]

Document(page_content='テキスト生成 AI利活用におけるリスクへの対策ガイ\nドブック（α版）  \n2024（令和6）年 5月30日 \nデジタル庁', metadata={'source': '/content/20240530_resources_generalitve-ai-guidebook_01.pdf', 'page': 0})

In [ ]:
# 日本語対応しているモデルを指定する
modelPath = "sentence-transformers/stsb-xlm-r-multilingual"

# GPUを有効にする
model_kwargs = {'device':'cuda'}

# エンコードオプションをOFFにします。 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# 上記のオプションを有効にしてHuggingFaceEmbeddingsを初期化します。
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
text = "This is a test document."
query_result = embeddings.embed_query(text)
query_result[:3]

[-0.1386953443288803, -0.1773158460855484, 0.7730242013931274]

In [ ]:
db = FAISS.from_documents(docs, embeddings)

In [ ]:
question = "テキスト生成 AIの利活用について説明しているページは？"
searchDocs = db.similarity_search(question)
print(searchDocs[0].page_content)

6 １.２ 本文書で想定するテキスト生成 AIの利活用方法  
 
本文書ではテキスト生成 AIを以下の 4つのユースケースを想定している。  
 
➢ ユースケース 1. チャットインターフェースで サービス 利用者とインタ
ラクティブに対話する機能としてテキスト生成 AIをオンライン処理で用
いる 
➢ ユースケース 2. 大量の文章に対してラベル付けやテキストデータ変換、
その他翻訳や要約や文章作成等の自然言語処理を行う機能としてテキス
ト生成AIをバッチ処理で用いる  
➢ ユースケース 3. 情報検索を目的としたサービスで検索エンジンの補助
としてテキスト生成 AIをオンライン処理で用いる  
➢ ユースケース 4. ダッシュボードやソースコード等を サービス 利用者の
自然言語により記述可能にする機能としてテキスト生成 AIをオンライン
処理で用いる  
 
テキスト生成 AIの利活用でもっとも有名な OpenAI社のChatGPT はこの類型
ではユースケース 1に該当する。テキスト生成 AIの利活用はチャット利用に限
定されない。  
ユースケース 2の一例は ２.２ ２） パブリックコメントの返答作成業務
にテキスト生成 AIを適切に利活用するユースケース で詳細に述べる。  
ユースケース 3の一例は ２．２  １） 情報検索サービスを国民向けに展開
する事例でテキスト生成 AIを適切に利活用する方法 で詳細に述べる。  
ユースケース 4の一例は ２．２  ４） 信頼できるデータベースと連携して
テキスト生成 AIを適切に利活用するユースケース で詳細に述べる。  
１.３ 本書の構成  
 
本文書は実践ガイドブックをベースにし、そこに対してテキスト生成 AI固有
のリスクや留意点を補足することを目指している。そこで、 「第３編第１章  実
践ガイドブックの構成  Point.2 チェックリスト」の分類にならい、新サービ
ス企画時、予算要求前、調達実施前、設計・開発時、サービス実施時の段階で
検討すべきテキスト生成 AI固有のリスクとその軽減策を述べる。  
１.４ 用語


In [ ]:
!pip install -q fugashi
!pip install -q unidic_lite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unidic_lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658818 sha256=1773fe63f45fa9f14fb8a20f3d04274d2dc295ceb31f22d4dd75eb852fbc3157
  Stored in directory: /root/.cache/pip/wheels/89/e8/68/f9ac36b8cc6c8b3c96888cd57434abed96595d444f42243853
Successfully built unidic_lite


In [ ]:
!pip install -q ipadic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 52.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
# 事前に学習されたモデルを読み込み、トークナイザーを作成する
tokenizer = AutoTokenizer.from_pretrained("tohoku-nlp/bert-base-japanese")

# 事前に学習されたBERTモデルを読み込み、質疑応答モデルを作成する
model = AutoModelForQuestionAnswering.from_pretrained("tohoku-nlp/bert-base-japanese")

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at tohoku-nlp/bert-base-japanese and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 使用するモデルを定義します。
model_name = "tohoku-nlp/bert-base-japanese"

# 指定された事前学習モデルのトークナイザーをロードします。
tokenizer = AutoTokenizer.from_pretrained(model_name, padding=True, truncation=True, max_length=512)

# モデルとトークナイザーを定義して質疑応答モデルのパイプラインを作成します。
question_answerer = pipeline(
    "question-answering",
    model=model_name,
    tokenizer=tokenizer,
    return_tensors='pt'
)

# HuggingFacePipelineのインスタンスを作成します。
llm = HuggingFacePipeline(
    pipeline=question_answerer,
    model_kwargs={"temperature": 0.7, "max_length": 512},
)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at tohoku-nlp/bert-base-japanese and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
retriever = db.as_retriever()

In [ ]:
# 最大4つの関連する分割/ドキュメントを取得する検索構成を使用して、「データベース」から取得オブジェクトを作成します。
retriever = db.as_retriever(search_kwargs={"k": 4})

# RetrievalQA クラスを使用して質問応答インスタンス (qa) を作成します。
# 言語モデル (LLM)、chain_type "refine" で構成します。
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever, return_source_documents=False)

In [ ]:
question = "生成AIを業務でのユースケースは?"
result = qa.run({"query": question})
print(result["result"])

ValueError: Context information is below. 
------------
39 ６.２ 非機能要件に関する実績データを把握しているか  
1) チャット利用の場合  
ここではユースケースのうち、 チャットインターフェースで利用者サービス
利用者とインタラクティブに対話する機能としてテキスト生成 AIをオンライン
処理で用いるケース について述べる。  
サービス利用者からの実際の入力がログの形で入手可能になり、それらがテ
ストケースに含まれていなかった場合、テストケースに追加することができる。
これは４．１  チャットでの利用 の期待品質  でも述べた。この実際の入力を
ベースとしたテストケースを用い、改めて品質評価を行うことで 非機能要件に
関する実績データが得られる。 ただし、 ６．１  業務の実績データや利用者
要望を分析できているか  で述べたように、重要な実績データである サービス
利用者からの入力文の扱いはプライバシー侵害だとみなされないように注意す
る。 
2) バッチ処理での利用の場合  
ここではユースケースのうち、 大量の文章に対してラベル付けやテキストデ
ータ変換、その他翻訳や要約や文章作成等の自然言語処理を行う機能としてテ
キスト生成 AIをバッチ処理で用いるケース について述べる。 ４．２  バッチ
処理での利用 の期待品質  で述べたテストケースを実績データから増やし、そ
れをテストすることで、非機能要件に関する実績データが得られる。  
3) 情報検索での利用の場合  
ここではユースケースのうち、 情報検索を目的としたサービスで検索エンジ
ンの補助としてテキスト生成 AIをオンライン処理で用いるケース につい述べる。
４．３  情報検索での利用 の期待品質  で述べたテストケースを実績データか
ら増やし、それをテストすることで、非機能要件に関する実績データが得られ
る。 
4) 自然言語からのプログラム作成での利用の場合  
ここではユースケースのうち、 ダッシュボードやソースコード等をサービス
利用者の自然言語により記述可能にする機能としてテキスト生成 AIをオンライ
ン処理で用いるケース について述べる。 ４．４  自然言語からのプログラム
作成での利用 の期待品質  で述べたテストケースを

In [ ]:
question = "生成AIの利用上のリスクは?"
result = qa.run({"query": question})
print(result["result"])

ValueError: Context information is below. 
------------
40 ６.３ 業務で扱うデータの品質を維持しつつ適切なライフサイクル管理を
行っているか  
 
一般的なシステム開発のデータ品質とライフサイクル管理はデータの不整合
の発生抑止に重視が置かれるが、ここでは AIプロジェクトの観点でのデータの
ライフサイクルについて述べる。これは、 (1)実際の利用 実績が当初想定から
乖離あるいは変化、 (2)利用した機械学習モデル (ここでは大規模言語モデル )
の変化、の 2つの変化により機能要件・非機能要件を満たさなくなるリスクの
ことを指す。機能要件が満たされているかの監視は一般的なシステム開発でも
共通した話であり、非機能要件が満たされているかの監視については ６．２  
非機能要件に関する実績データを把握しているか  で述べたので割愛する。  
機能要件・非機能要件が満たされなくなる前に気が付く、満たされなくなっ
た後に原因分析を行い適切な対策をとる、の両方のために、原因となる変化を
観測することは非常に重要である。仮にシステムが機能要件・非機能要件を満
たさなくなった場合、その要件を再び満たすための改修だけでなく、サービス
の停止や終了も視野にいれた検討も視野に入れるべきである。  
(1)実際の利用実績が当初想定から乖離あるいは変化したか、についての観
測は６．１  業務の実績データや利用者要望を分析できているか  で述べた。
特にサービス実施直後は、設計・開発時との想定との乖離がないかをしっかり
と確認することを推奨する。  
(2)利用した機械学習モデル (ここでは大規模言語モデル )の変化については、
テキスト生成 AIの機械学習モデルが直接提供されるケース以外の利用形態につ
いては注意すべき点であり、付録の ７．１ ４）アップデート対応に関する
リスクと７．２．４） アップデート対応に関するリスク を参照していただき
たい。  
６.４ ６章 まとめ 
サービス実施時には、実際の運用データをもとにした継続的な品質評価が必
要である。本章では、業務実績データの分析と非機能要件の達成度の検証に焦
点を当て、適切なライフサイクル管理を通じて、サービスの持続可能性を保証
するための留意点を強調した。これら